In [ ]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MOUNT_DIR = '/content/drive/MyDrive/BarclaysReviews/'
DATA_DIR = '/content/drive/MyDrive/BarclaysReviews/data/'
MODEL_DIR = '/content/drive/MyDrive/BarclaysReviews/model/'


In [ ]:
def saveFileToDrive(path,dataframe):

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    dataframe.to_csv(f)
    print("Saving file : "+path)

In [ ]:
final_merged_reviews = pd.read_csv(DATA_DIR+"final_merged_reviews_cleaned.csv")

In [ ]:
pd.set_option('display.max_colwidth', -1)
# final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("reward")][["review_description", "source","rating"]].rating.value_counts()
final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("blue")][["review_description", "source","rating"]].shape

(49, 3)

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
barclays_stop_list = ['app', 'barclays','need','will','pay','work']
stop_words.extend(barclays_stop_list)
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

import re
wnl = WordNetLemmatizer()
def tokenizer(text):
    regex_num_ponctuation = '(\d+)|([^\w\s])'
    regex_little_words = r'(\b\w{1,2}\b)'
    return [wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct ]

def LemmaTokenizer(doc):
      wnl = WordNetLemmatizer()
      regex_num_ponctuation = '(\d+)|([^\w\s])'
      regex_little_words = r'(\b\w{1,2}\b)'
      return [wnl.lemmatize(t) for t in word_tokenize(doc) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t) and not t.is_stop and not t.is_punct]

In [ ]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
from bertopic import BERTopic

In [ ]:
import time

In [ ]:
def topicModellingBERT(df,columns, textCol = "cleaned_data",min_df=4,max_features = 10000,min_cluster_size=80,top_n_words=5,path="",model_name="",replace_dict={}):
  print(columns)
  if bool(columns):
    # print("columns")
    for key in columns:
      # print(df.shape)
      df = df[df[key].str.contains(columns[key])]

  umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
  hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)
  min_df = min_df # minimum required occurences of a word, e.g 4
  max_features = max_features # max number of unique words, e.g 10000

  # vectorizer_model = CountVectorizer(
  #               min_df=min_df, 
  #               max_features=max_features, 
  #               tokenizer=tokenizer, # use tokenizer we defined before
  #               ngram_range=(1,2),
  #               stop_words=stop_words) # we use unigram & bigram
  # X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])


  df = df[df[textCol].notnull()]
  if bool(replace_dict):
    df[textCol].replace(replace_dict,inplace=True,regex=True) 
  print('dataet length: ',df.shape,'\n')
  # # we add this to remove stopwords that can pollute topcs
  vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=min_df,max_features=max_features)
  df.reset_index(drop=True, inplace=True)
  model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
  )
  topics, probs = model.fit_transform(df[textCol])
  print(model.get_topic_info())

  custom_labels = []
  custom_labels_temp = model.generate_topic_labels(nr_words=1)
  # negative_custom_labels.type
  for i in custom_labels_temp:
    custom_labels.append(i.split('_')[1])
  model.set_topic_labels(custom_labels)

  model.save(path+model_name+'_'+str(int(time.time())))
  return  df,topics, probs, model



In [ ]:
#  topics, probs, model = topicModellingBERT(df=final_merged_reviews,sentiment_col="customer_sentiment",sentiment='NEGATIVE',path=MODEL_DIR,model_name='negative_topic_modelling')
 

In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_df, negative_topics,  negative_probs,  negative_model = topicModellingBERT(df=df,columns = {"CustomName":'contactless'}, min_df=2,min_cluster_size=80,path=MODEL_DIR,model_name='negative_topic_1_modelling',replace_dict={"contactless":"","payment":"","slow":"","crash":"","account":""})
 

{'CustomName': 'contactless'}
dataet length:  (1246, 28) 



Batches:   0%|          | 0/39 [00:00<?, ?it/s]

2023-05-01 10:31:22,681 - BERTopic - Transformed documents to Embeddings
2023-05-01 10:31:26,913 - BERTopic - Reduced dimensionality
2023-05-01 10:31:27,013 - BERTopic - Clustered reduced embeddings


   Topic  Count                                  Name
0 -1      660    -1_phone_device_bad_update          
1  0      238    0_googlepay_card_login_device       
2  1      162    1_transfer_money_phone_card         
3  2      94     2_barclaycard_credicard_barclay_card
4  3      92     3_update_device_phone_loginin       


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_Account_df, negative_Account_topics,  negative_Account_probs,  negative_Account_model = topicModellingBERT(df=df,columns = {"CustomName":'account'} ,min_df=2,min_cluster_size=40,path=MODEL_DIR,model_name='negative_topic_account_modelling',replace_dict={"account":"","slow":""})


{'CustomName': 'account'}
dataet length:  (604, 28) 



Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-05-01 10:22:08,355 - BERTopic - Transformed documents to Embeddings
2023-05-01 10:22:11,453 - BERTopic - Reduced dimensionality
2023-05-01 10:22:11,498 - BERTopic - Clustered reduced embeddings


   Topic  Count                                 Name
0 -1      15     -1_saving_lloyd_latest_version_save
1  0      328    0_money_bad_year_payment           
2  1      261    1_login_open_register_device       


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_crash_df, negative_crash_topics,  negative_crash_probs,  negative_crash_model = topicModellingBERT(df=df,columns = {"CustomName":'crash'} ,min_df=2,min_cluster_size=40,path=MODEL_DIR,model_name='negative_topic_crash_modelling',replace_dict={"crash":"","slow":""})


{'CustomName': 'crash'}
dataet length:  (496, 28) 



Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2023-05-01 10:16:55,978 - BERTopic - Transformed documents to Embeddings
2023-05-01 10:16:58,957 - BERTopic - Reduced dimensionality
2023-05-01 10:16:58,994 - BERTopic - Clustered reduced embeddings


   Topic  Count                                       Name
0 -1      139    -1_constantly_notification_calendar_fix  
1  0      156    0_calendar_view_payment_calendar_calender
2  1      119    1_notification_phone_ped_worke_bad       
3  2      82     2_login_bad_review_constantly            


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_chat_df, negative_chat_topics,  negative_chat_probs,  negative_chat_model = topicModellingBERT(df=df,columns = {"CustomName":'chat'} ,min_df=1,min_cluster_size=5,path=MODEL_DIR,model_name='negative_topic_chat_modelling',replace_dict={"chat":""})


{'CustomName': 'chat'}
dataet length:  (423, 28) 



Batches:   0%|          | 0/14 [00:00<?, ?it/s]

2023-05-01 10:10:15,943 - BERTopic - Transformed documents to Embeddings
2023-05-01 10:10:19,982 - BERTopic - Reduced dimensionality
2023-05-01 10:10:20,015 - BERTopic - Clustered reduced embeddings


   Topic  Count                                       Name
0 -1      334    -1_slow_help_bad_service                 
1  0      79     0_slow_service_bad_customer              
2  1      5      1_download statement_statement_page_today
3  2      5      2_someslow_login_payment_login everyslow 


In [ ]:
negative_slow_model.visualize_barchart()

In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_slow_df, negative_slow_topics,  negative_slow_probs,  negative_slow_model = topicModellingBERT(df=df,columns = {"CustomName":'slow'} ,min_df=2,min_cluster_size=80,path=MODEL_DIR,model_name='negative_topic_slow_modelling',replace_dict={"slow":""})


{'CustomName': 'slow'}
dataet length:  (2243, 28) 



Batches:   0%|          | 0/71 [00:00<?, ?it/s]

2023-05-01 11:43:05,264 - BERTopic - Transformed documents to Embeddings
2023-05-01 11:43:14,783 - BERTopic - Reduced dimensionality
2023-05-01 11:43:14,964 - BERTopic - Clustered reduced embeddings


    Topic  Count                                                 Name
0  -1      766    -1_bad_card_phone_update                           
1   0      996    0_account_payment_phone_login                      
2   1      105    1_googlepay_support_support googlepay_google_wallet
3   2      82     2_bad_bad bad_awful_bad awful                      
4   3      64     3_useless_trash_device_android                     
5   4      62     4_open_advert_update_open open                     
6   5      46     5_service_customer_customer service_bad            
7   6      36     6_nightmare_inconsistent_unreliable_unstable       
8   7      34     7_download_update_statement_download statement     
9   8      32     8_load_load load_open load_open                    
10  9      20     9_bad_money_trouble_company                        


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_notification_df, negative_notification_topics,  negative_notification_probs,  negative_notification_model = topicModellingBERT(df=df,columns = {"CustomName":'notification'} ,min_df=1,min_cluster_size=5,path=MODEL_DIR,model_name='negative_topic_notification_modelling',replace_dict={"notification":""})


{'CustomName': 'notification'}
dataet length:  (103, 28) 



Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2023-05-01 09:59:28,476 - BERTopic - Transformed documents to Embeddings
2023-05-01 09:59:32,690 - BERTopic - Reduced dimensionality
2023-05-01 09:59:32,704 - BERTopic - Clustered reduced embeddings


   Topic  Count                             Name
0 -1      103    -1_payment_money_account_online


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_cheque_df, negative_cheque_topics,  negative_cheque_probs,  negative_cheque_model = topicModellingBERT(df=df,columns = {"CustomName":'cheque'} ,min_df=1,min_cluster_size=5,path=MODEL_DIR,model_name='negative_topic_cheque_modelling',replace_dict={"cheque":""})


{'CustomName': 'cheque'}
dataet length:  (463, 28) 



Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2023-05-01 09:57:38,703 - BERTopic - Transformed documents to Embeddings
2023-05-01 09:57:42,176 - BERTopic - Reduced dimensionality
2023-05-01 09:57:42,241 - BERTopic - Clustered reduced embeddings


   Topic  Count                         Name
0 -1      243    -1_slow_photo_bad_scan     
1  0      202    0_photo_slow_bad_picture   
2  1      18     1_branch_business_slow_tell


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
negative_update_df, negative_update_topics,  negative_update_probs,  negative_update_model = topicModellingBERT(df=df,columns = {"CustomName":'update'} ,min_df=1,min_cluster_size=5,path=MODEL_DIR,model_name='negative_topic_update_modelling',replace_dict={"update":""})


{'CustomName': 'update'}
dataet length:  (471, 28) 



Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2023-05-01 11:13:34,110 - BERTopic - Transformed documents to Embeddings
2023-05-01 11:13:38,243 - BERTopic - Reduced dimensionality
2023-05-01 11:13:38,281 - BERTopic - Clustered reduced embeddings


   Topic  Count                                     Name
0 -1      324    -1_slow_phone_bad_account              
1  0      69     0_crash_slow_open_great                
2  1      40     1_slow_change_bad_access               
3  2      20     2_phone_new_login_reinstall            
4  3      13     3_force_upload_problem_bad             
5  4      5      4_access_auto_google_pixel_access money


In [ ]:
# df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
# negative_navigate_df, negative_navigate_topics,  negative_navigate_probs,  negative_navigate_model = topicModellingBERT(df=df,columns = {"CustomName":'navigate'} ,min_df=1,min_cluster_size=5,path=MODEL_DIR,model_name='negative_topic_navigate_modelling')
negative_navigate_model = BERTopic.load(MODEL_DIR+'negative_topic_navigate_modelling_1682939644')
negative_navigate_model.get_topic_info()

,Topic,Count,Name,CustomName
0,-1,28,-1_phone_android_loginge_welcome,phone
1,0,53,0_crash_account_open_fscs,crash
2,1,42,1_statement_document_slow_simple,statement


In [ ]:
df = pd.read_csv(DATA_DIR+"Negative_topic_modelling_data.csv")
pd.set_option('display.max_colwidth', -1)
df[df["cleaned_data"].notnull() & df["cleaned_data"].str.contains("mortgage")][["review_description", "CustomName","source","rating"]]

,review_description,CustomName,source,rating
2,bad experience - error messages despite being up most recent version. not showing my mortgage on the app. limiting my option as to which account to open. not stable at all,slow,Google Play,1
13,slow compared to other banking apps. fundamental functions missing that are supposed to be there. trying to edit monthly mortgage payments but you can't edit monthly payments on the app. app does not contain 'pinsentry' button required to loginin on browser. i can't create a current account because it auto logins me into my mortgage. it closes itself whenever i change to a different app (which i need to do to verify payment from another bank). it is not suitable for mortgage customers.,slow,Google Play,1
191,app has lost my mortgage and now reverse ed to a former address. not sure how well barclays are managing my personal data now.,slow,Google Play,1
269,"it's a nothing special banking app, only doing what every banking app does. one star for insulting our intelligence by telling us it gives us rewards when, in fact, they aren't rewards they are marketing. boots points are rewards, meerkat meals are rewards. 25% off a subscription for a month, then full price is just marketing. as for blue rewards, pay £5 and get £5 back. it is pointless. why buy your mortgage or insurance products when others are cheaper even with your offers? no respect.",notification,Google Play,1
329,"since the last update the app keeps closing down on me while i'm using it. it happened on few customer journeys like checking balance, trying to contact customer service, checking mortgage balance etc. it's so bad. the contact less payments are not reliable for me too so haven't used it for a while now due to being mega embarrassing. if you were supporting at least samsung pay, it would make things so much easier for samsung users.",slow,Google Play,2
...,...,...,...,...
5875,credicard and mortgages transactions display normally. current account is a single column and looks awful. your suggested ‘fix’ is to change my settings to alter the way my entire phone looks. if i do that everything else is too small. there is a reason some of us have to use large text. please allow the current account to display like the other accounts.,slow,App Store,2
5997,"i’m very annoyed at the app and barclays ,it’s so hard to get an up to date statement .. then you have to slow in line on the chat for so slow . and the phone slow is always 20 minutes to india … enter the branch and have sore back from queuing ,and lose 1 say off you life … after i get my mortgage,i’m switch appsing back to the new american bank with the good hours … \n\nthis new wave of bank making us use apps and machines in the walls are horrible . zero stars",chat,App Store,1
6022,this app has not been thought thru for mortgage only customers. taken a week to get registered including a 45 minute phone call slow. they lack the imagination to consider you having a mortgage without their current account. it requires your phone number which is not collected during mortgage application. does not allow you to make lump sum overpayments or set up recurring dd payments. virtually pointless and very badly thought out,slow,App Store,1
6059,"1) i got a mortgage worth barclays and installed the app. i could see some functionality but was told i needed to open a current account to get full functionality (eg changing direct debit collection dates)\n\n2) i opened a current account. \n\n3) my current account did not appear in my app. ok, i think, i’ll remove the app and reinstall. \n\n4) i re-installed the app and plugged in my new current account details … but now there is no sign of the mortgage. there’s no option to link the current account and the mortgage, off ed at least there isn’t one i can see. \n\n5) faqs don’t help, and there’s no obvious online support. \n\ni can’t be the only one with this problem, right guys? \n\nplease help!",slow,App Store,1
